In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [2]:
import xlnet
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import model_utils

/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

/home/husein/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorboard/compat/ten

In [3]:
from prepro_utils import preprocess_text, encode_ids

def tokenize_fn(text):
    text = preprocess_text(text, lower= False)
    return encode_ids(sp_model, text)

In [7]:
# !wget https://github.com/huseinzol05/Malaya/raw/master/pretrained-model/preprocess/sp10m.cased.v9.model

In [6]:
import sentencepiece as spm
from prepro_utils import preprocess_text, encode_ids

sp_model = spm.SentencePieceProcessor()
sp_model.Load('sp10m.cased.v9.model')

True

In [8]:
from glob import glob
import json

left, right, label = [], [], []
for file in glob('../text-similarity/*k.json'):
    with open(file) as fopen:
        x = json.load(fopen)
    for i in x:
        splitted = i[0].split(' <> ')
        if len(splitted) != 2:
            continue
        left.append(splitted[0])
        right.append(splitted[1])
        label.append(i[1])

In [9]:
list(zip(left[:10], right[:10], label[:10]))

[('Bagaimanakah saya boleh menjadi ahli geologi yang baik?',
  'Apa yang perlu saya lakukan untuk menjadi ahli geologi yang hebat?',
  1),
 ('Sekiranya saya membeli tiago?',
  'Apa yang menjaga kanak-kanak aktif dan jauh dari telefon dan permainan video?',
  0),
 ('Motorola (syarikat): Bolehkah saya menggodam Piagam Motorolla DCX3400 saya?',
  'Bagaimana saya mengesan Motorola DCX3400 untuk internet percuma?',
  0),
 ('Apakah kisah Kohinoor (Koh-i-Noor) Diamond?',
  'Apa yang akan berlaku jika kerajaan India mencuri kembali berlian Kohinoor (Koh-i-Noor)?',
  0),
 ('Apakah panduan langkah demi langkah untuk melabur dalam pasaran saham di india?',
  'Apakah panduan langkah demi langkah untuk melabur dalam pasaran saham?',
  0),
 ('Bilakah anda menggunakan シ bukan し?',
  'Bilakah anda menggunakan "&" bukan "dan"?',
  0),
 ('Astrologi: Saya Capricorn Sun Cap moon dan cap naik ... apa kata itu tentang saya?',
  'Saya Capricorn tiga kali ganda (Sun, Moon dan naik di Capricorn) Apa kata ini t

In [10]:
l = {'contradiction': 0, 'entailment': 1}


snli = glob('../text-similarity/part*.json')
for file in snli:
    with open(file) as fopen:
        x = json.load(fopen)
    for i in x:
        splitted = i[1].split(' <> ')
        if len(splitted) != 2:
            continue
        if i[0] not in l:
            continue
        left.append(splitted[0])
        right.append(splitted[1])
        try:
            label.append(l[i[0]])
        except Exception as e:
            print(e)
            print(splitted, i[0])

In [11]:
list(zip(left[-10:], right[-10:], label[-10:]))

[('Seorang wanita tidur di bawah selimut biru dan putih di dalam kereta.',
  'Seorang wanita tidur di dalam kereta.',
  1),
 ('Sesetengah orang di dalam air dan seorang yang berdiri di atas papan.',
  'Orang-orang di pantai.',
  1),
 ('Sesetengah orang di dalam air dan seorang yang berdiri di atas papan.',
  'Orang ramai sedang bersiap untuk tidur.',
  0),
 ('Lelaki dalam pakaian polis bersebelahan seorang lelaki dalam kostum zombie.',
  'Wanita dalam pakaian polis bersebelahan dengan seorang lelaki dalam kostum zombie.',
  0),
 ('Lelaki dalam pakaian polis bersebelahan seorang lelaki dalam kostum zombie.',
  'Seseorang dalam pakaian polis bersebelahan seorang lelaki dalam kostum zombie.',
  1),
 ('Seorang lelaki dalam pakaian seragam berdiri di sebelah seorang lelaki dalam kostum seram.',
  'Lelaki duduk di sebelah wanita.',
  0),
 ('Di sebalik dermaga yang panjang ke dalam tasik, beberapa orang berenang dan menikmati air.',
  'Cuaca sangat buruk di luar.',
  0),
 ('Seorang wanita tid

In [12]:
mnli = glob('../text-similarity/translated-*.json')
mnli

['../text-similarity/translated-1.json',
 '../text-similarity/translated-4.json',
 '../text-similarity/translated-3.json',
 '../text-similarity/translated-0.json',
 '../text-similarity/translated-2.json']

In [13]:
for file in mnli:
    with open(file) as fopen:
        x = json.load(fopen)
    for i in x:
        if len(i) != 3:
            continue
        splitted = i[2].split(' <> ')
        if len(splitted) != 3:
            continue
        if i[1] not in l:
            continue
        left.append(splitted[0])
        right.append(splitted[1])
        try:
            label.append(l[i[1]])
        except Exception as e:
            print(e)
            print(splitted, i)

In [14]:
list(zip(left[-10:], right[-10:], label[-10:]))

[('Vista yang mengagumkan ini pada awalnya dirancang untuk dilihat oleh Napo Leon dari bilik tidurnya di Louvre, yang pada masa itu adalah istana.',
  'Pemandangan monumental dibina untuk dilihat oleh Napoleon dari bilik tidurnya di Louvre.',
  1),
 ('Ia menggeram dan tersentak di udara, dan walaupun berjaya menghentak satu korban (yang, dengan sentuhan yang bagus, melekat di bahagian bawah kakinya seperti sepotong permen karet), secara amnya ia bergerak dengan gaya berjalan yang berat sehingga kita mungkin juga kembali pada tahun 60-an menyaksikan Lembah Gwangi. Di manakah kelajuan maut pemangsa?',
  'Pemangsa tidak pernah dapat membunuh mangsa.',
  0),
 ('Republik Turki moden bermula pada tahun 1923, tetapi sejarah tanah di dalam sempadannya bermula sejak awal umat manusia.',
  'Turki mempunyai sejarah panjang sejak ribuan tahun.',
  0),
 ('Kawasan membeli-belah utama adalah Tsim Sha Tsui di Kowloon, terutama di sepanjang Jalan Nathan; Pusat di Pulau Hong Kong, terutama untuk barang 

In [15]:
SEG_ID_A   = 0
SEG_ID_B   = 1
SEG_ID_CLS = 2
SEG_ID_SEP = 3
SEG_ID_PAD = 4

special_symbols = {
    "<unk>"  : 0,
    "<s>"    : 1,
    "</s>"   : 2,
    "<cls>"  : 3,
    "<sep>"  : 4,
    "<pad>"  : 5,
    "<mask>" : 6,
    "<eod>"  : 7,
    "<eop>"  : 8,
}

VOCAB_SIZE = 32000
UNK_ID = special_symbols["<unk>"]
CLS_ID = special_symbols["<cls>"]
SEP_ID = special_symbols["<sep>"]
MASK_ID = special_symbols["<mask>"]
EOD_ID = special_symbols["<eod>"]
MAX_SEQ_LENGTH = 150

In [16]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
              tokens_a.pop()
        else:
              tokens_b.pop()
                
def get_inputs(left, right):

    input_ids, input_mask, all_seg_ids = [], [], []
    
    for i in tqdm(range(len(left))):
        tokens = tokenize_fn(left[i])
        tokens_right = tokenize_fn(right[i])
        _truncate_seq_pair(tokens, tokens_right, MAX_SEQ_LENGTH - 3)
        
        segment_ids = [SEG_ID_A] * len(tokens)
        tokens.append(SEP_ID)
        segment_ids.append(SEG_ID_A)

        tokens.extend(tokens_right)
        segment_ids.extend([SEG_ID_B] * len(tokens_right))
        tokens.append(SEP_ID)
        segment_ids.append(SEG_ID_B)

        tokens.append(CLS_ID)
        segment_ids.append(SEG_ID_CLS)
        cur_input_ids = tokens
        cur_input_mask = [0] * len(cur_input_ids)
        input_ids.append(tokens)
        input_mask.append(cur_input_mask)
        all_seg_ids.append(segment_ids)
        
    return input_ids, input_mask, all_seg_ids

In [17]:
input_ids, input_masks, segment_ids = get_inputs(left, right)

100%|██████████| 1041526/1041526 [02:52<00:00, 6044.58it/s]


In [18]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

input_ids = pad_sequences(input_ids,padding='post')
input_masks = pad_sequences(input_masks,padding='post', value = 1)
segment_ids = pad_sequences(segment_ids,padding='post', value = 4)

In [20]:
kwargs = dict(
      is_training=True,
      use_tpu=False,
      use_bfloat16=False,
      dropout=0.1,
      dropatt=0.1,
      init='normal',
      init_range=0.1,
      init_std=0.05,
      clamp_len=-1)

xlnet_parameters = xlnet.RunConfig(**kwargs)
xlnet_config = xlnet.XLNetConfig(json_path='alxlnet-base-2020-04-10/config.json')

In [21]:
epoch = 20
batch_size = 60
warmup_proportion = 0.1
num_train_steps = int(len(input_ids) / batch_size * epoch)
num_warmup_steps = int(num_train_steps * warmup_proportion)
print(num_train_steps, num_warmup_steps)
learning_rate = 2e-5

training_parameters = dict(
      decay_method = 'poly',
      train_steps = num_train_steps,
      learning_rate = learning_rate,
      warmup_steps = num_warmup_steps,
      min_lr_ratio = 0.0,
      weight_decay = 0.00,
      adam_epsilon = 1e-8,
      num_core_per_host = 1,
      lr_layer_decay_rate = 1,
      use_tpu=False,
      use_bfloat16=False,
      dropout=0.0,
      dropatt=0.0,
      init='normal',
      init_range=0.1,
      init_std=0.05,
      clip = 1.0,
      clamp_len=-1,)

347175 34717


In [22]:
class Parameter:
    def __init__(self, decay_method, warmup_steps, weight_decay, adam_epsilon, 
                num_core_per_host, lr_layer_decay_rate, use_tpu, learning_rate, train_steps,
                min_lr_ratio, clip, **kwargs):
        self.decay_method = decay_method
        self.warmup_steps = warmup_steps
        self.weight_decay = weight_decay
        self.adam_epsilon = adam_epsilon
        self.num_core_per_host = num_core_per_host
        self.lr_layer_decay_rate = lr_layer_decay_rate
        self.use_tpu = use_tpu
        self.learning_rate = learning_rate
        self.train_steps = train_steps
        self.min_lr_ratio = min_lr_ratio
        self.clip = clip
        
training_parameters = Parameter(**training_parameters)

In [23]:
class Model:
    def __init__(
        self,
        dimension_output,
        learning_rate = 2e-5,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.segment_ids = tf.placeholder(tf.int32, [None, None])
        self.input_masks = tf.placeholder(tf.float32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None])
        
        xlnet_model = xlnet.XLNetModel(
            xlnet_config=xlnet_config,
            run_config=xlnet_parameters,
            input_ids=tf.transpose(self.X, [1, 0]),
            seg_ids=tf.transpose(self.segment_ids, [1, 0]),
            input_mask=tf.transpose(self.input_masks, [1, 0]))
        
        summary = xlnet_model.get_pooled_out("last", True)
        print(summary)
        
        self.logits = tf.layers.dense(summary, dimension_output)
        self.logits = tf.identity(self.logits, name = 'logits')
        
        self.cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        
        self.optimizer, self.learning_rate, _ = model_utils.get_train_op(training_parameters, self.cost)
        
        correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [24]:
dimension_output = 2

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate
)

sess.run(tf.global_variables_initializer())




INFO:tensorflow:memory input None
INFO:tensorflow:Use float type <dtype: 'float32'>

Instructions for updating:
Use keras.layers.dropout instead.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.dense instead.
Tensor("model_1/sequnece_summary/dropout/dropout/mul_1:0", shape=(?, 768), dtype=float32)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [25]:
import collections
import re

def get_assignment_map_from_checkpoint(tvars, init_checkpoint):
    """Compute the union of the current variables and checkpoint variables."""
    assignment_map = {}
    initialized_variable_names = {}

    name_to_variable = collections.OrderedDict()
    for var in tvars:
        name = var.name
        m = re.match('^(.*):\\d+$', name)
        if m is not None:
            name = m.group(1)
        name_to_variable[name] = var

    init_vars = tf.train.list_variables(init_checkpoint)

    assignment_map = collections.OrderedDict()
    for x in init_vars:
        (name, var) = (x[0], x[1])
        if name not in name_to_variable:
            continue
        assignment_map[name] = name_to_variable[name]
        initialized_variable_names[name] = 1
        initialized_variable_names[name + ':0'] = 1

    return (assignment_map, initialized_variable_names)

In [26]:
tvars = tf.trainable_variables()
checkpoint = 'alxlnet-base-2020-04-10/model.ckpt-300000'
assignment_map, initialized_variable_names = get_assignment_map_from_checkpoint(tvars, 
                                                                                checkpoint)

In [27]:
saver = tf.train.Saver(var_list = assignment_map)
saver.restore(sess, checkpoint)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from alxlnet-base-2020-04-10/model.ckpt-300000


In [28]:
from sklearn.model_selection import train_test_split

train_input_ids, test_input_ids, train_input_masks, test_input_masks, train_segment_ids, test_segment_ids, train_Y, test_Y = train_test_split(
    input_ids, input_masks, segment_ids, label, test_size = 0.2)

In [29]:
from tqdm import tqdm
import time

EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 1, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = [], [], [], []
    pbar = tqdm(
        range(0, len(train_input_ids), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_input_ids))
        batch_x = train_input_ids[i: index]
        batch_masks = train_input_masks[i: index]
        batch_segment = train_segment_ids[i: index]
        batch_y = train_Y[i: index]
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.segment_ids: batch_segment,
                model.input_masks: batch_masks
            },
        )
        assert not np.isnan(cost)
        train_loss.append(cost)
        train_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
    pbar = tqdm(range(0, len(test_input_ids), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_input_ids))
        batch_x = test_input_ids[i: index]
        batch_masks = test_input_masks[i: index]
        batch_segment = test_segment_ids[i: index]
        batch_y = test_Y[i: index]
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.segment_ids: batch_segment,
                model.input_masks: batch_masks
            },
        )
        test_loss.append(cost)
        test_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)

    train_loss = np.mean(train_loss)
    train_acc = np.mean(train_acc)
    test_loss = np.mean(test_loss)
    test_acc = np.mean(test_acc)

    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   0%|          | 0/13887 [00:00<?, ?it/s]

epoch: 0, pass acc: 0.000000, current acc: 0.812083
time taken: 14661.727264642715
epoch: 0, training loss: 0.552618, training acc: 0.710324, valid loss: 0.408168, valid acc: 0.812083



train minibatch loop:   0%|          | 0/13887 [00:00<?, ?it/s]

epoch: 1, pass acc: 0.812083, current acc: 0.850941
time taken: 14621.280127048492
epoch: 1, training loss: 0.367243, training acc: 0.835550, valid loss: 0.335863, valid acc: 0.850941



train minibatch loop:   0%|          | 0/13887 [00:00<?, ?it/s]

...

epoch: 6, pass acc: 0.882793, current acc: 0.883201
time taken: 14608.575321674347
epoch: 6, training loss: 0.193508, training acc: 0.923616, valid loss: 0.300480, valid acc: 0.883201



train minibatch loop:  38%|███▊      | 5325/13887 [1:25:45<2:17:19,  1.04it/s, accuracy=0.917, cost=0.205] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

train minibatch loop:  68%|██████▊   | 9399/13887 [2:31:10<1:12:04,  1.04it/s, accuracy=0.9, cost=0.168]   IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

train minibatch loop:  96%|█████████▋| 13368/13887 [3:34:53<08:19,  1.04it/s, accuracy=0.9, cost=0.232]   IOPub message rate exceeded.
T

epoch: 8, pass acc: 0.883243, current acc: 0.884002
time taken: 14597.953269004822
epoch: 8, training loss: 0.156109, training acc: 0.940389, valid loss: 0.330420, valid acc: 0.884002



test minibatch loop: 100%|██████████| 3472/3472 [19:39<00:00,  2.94it/s, accuracy=0.935, cost=0.214] 

time taken: 14638.857108354568
epoch: 9, training loss: 0.141414, training acc: 0.947097, valid loss: 0.343461, valid acc: 0.883103

break epoch:10



In [30]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'alxlnet-base-similarity/model.ckpt')

'alxlnet-base-similarity/model.ckpt'

In [31]:
kwargs = dict(
      is_training=False,
      use_tpu=False,
      use_bfloat16=False,
      dropout=0.0,
      dropatt=0.0,
      init='normal',
      init_range=0.1,
      init_std=0.05,
      clamp_len=-1)

xlnet_parameters = xlnet.RunConfig(**kwargs)
xlnet_config = xlnet.XLNetConfig(json_path='alxlnet-base-2020-04-10/config.json')

In [32]:
dimension_output = 2
learning_rate = 2e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate
)

sess.run(tf.global_variables_initializer())

INFO:tensorflow:memory input None
INFO:tensorflow:Use float type <dtype: 'float32'>


/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Tensor("model_1/sequnece_summary/dropout/Identity:0", shape=(?, 768), dtype=float32)


In [33]:
saver = tf.train.Saver(tf.trainable_variables())
saver.restore(sess, 'alxlnet-base-similarity/model.ckpt')

INFO:tensorflow:Restoring parameters from alxlnet-base-similarity/model.ckpt


In [34]:
real_Y, predict_Y = [], []

pbar = tqdm(
    range(0, len(test_input_ids), batch_size), desc = 'validation minibatch loop'
)
for i in pbar:
    index = min(i + batch_size, len(test_input_ids))
    batch_x = test_input_ids[i: index]
    batch_masks = test_input_masks[i: index]
    batch_segment = test_segment_ids[i: index]
    batch_y = test_Y[i: index]
    predict_Y += np.argmax(sess.run(model.logits,
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.segment_ids: batch_segment,
                model.input_masks: batch_masks
            },
    ), 1, ).tolist()
    real_Y += batch_y

validation minibatch loop: 100%|██████████| 3472/3472 [17:01<00:00,  3.40it/s]


In [35]:
from sklearn import metrics

print(
    metrics.classification_report(
        real_Y, predict_Y, target_names = ['not similar', 'similar'],
        digits = 5
    )
)

              precision    recall  f1-score   support

 not similar    0.89614   0.90170   0.89891    114554
     similar    0.87897   0.87231   0.87563     93752

    accuracy                        0.88847    208306
   macro avg    0.88756   0.88700   0.88727    208306
weighted avg    0.88841   0.88847   0.88843    208306



In [36]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'logits' in n.name
        or 'alphas' in n.name
        or 'self/Softmax' in n.name)
        and 'Adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
    ]
)
strings.split(',')

['Placeholder',
 'Placeholder_1',
 'Placeholder_2',
 'Placeholder_3',
 'model/transformer/r_w_bias',
 'model/transformer/r_r_bias',
 'model/transformer/word_embedding/lookup_table',
 'model/transformer/word_embedding/lookup_table_2',
 'model/transformer/r_s_bias',
 'model/transformer/seg_embed',
 'model/transformer/layer_shared/rel_attn/q/kernel',
 'model/transformer/layer_shared/rel_attn/k/kernel',
 'model/transformer/layer_shared/rel_attn/v/kernel',
 'model/transformer/layer_shared/rel_attn/r/kernel',
 'model/transformer/layer_shared/rel_attn/o/kernel',
 'model/transformer/layer_shared/rel_attn/LayerNorm/gamma',
 'model/transformer/layer_shared/ff/layer_1/kernel',
 'model/transformer/layer_shared/ff/layer_1/bias',
 'model/transformer/layer_shared/ff/layer_2/kernel',
 'model/transformer/layer_shared/ff/layer_2/bias',
 'model/transformer/layer_shared/ff/LayerNorm/gamma',
 'model/sequnece_summary/summary/kernel',
 'model/sequnece_summary/summary/bias',
 'dense/kernel',
 'dense/bias',
 '

In [37]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [38]:
freeze_graph('alxlnet-base-similarity', strings)

INFO:tensorflow:Restoring parameters from alxlnet-base-similarity/model.ckpt
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 23 variables.
INFO:tensorflow:Converted 23 variables to const ops.
7392 ops in the final graph.
